In [17]:
import findspark
findspark.init()
import pyspark
import datetime
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE, N3, CSV, XML

In [2]:
sc = pyspark.SparkContext("local[*]",appName = "Freebase_people")

In [3]:
subor = "freebase-head-10000000"

In [4]:
osoby = sc.textFile(subor).filter(lambda x: 'type.object.name' in x or 'common.topic.alias' in x or 'people.person.date_of_birth' in x or 'people.deceased_person.date_of_death' in x).distinct()

In [19]:
osoby.take(100)

['<http://rdf.freebase.com/ns/base.collectives.collective_member>\t<http://rdf.freebase.com/ns/type.object.name>\t"Члены коллектива"@ru\t.',
 '<http://rdf.freebase.com/ns/base.collectives.collective_member>\t<http://rdf.freebase.com/ns/type.object.name>\t"Člen spolku"@sk\t.',
 '<http://rdf.freebase.com/ns/base.crmbase.developer>\t<http://rdf.freebase.com/ns/type.object.name>\t"developer"@en\t.',
 '<http://rdf.freebase.com/ns/base.horsefacts.tack>\t<http://rdf.freebase.com/ns/type.object.name>\t"마구"@ko\t.',
 '<http://rdf.freebase.com/ns/base.horsefacts.tack>\t<http://rdf.freebase.com/ns/type.object.name>\t"ιπποσκευή"@el\t.',
 '<http://rdf.freebase.com/ns/base.mediaasset.provider.parent>\t<http://rdf.freebase.com/ns/type.object.name>\t"Parent"@en\t.',
 '<http://rdf.freebase.com/ns/base.myevents.project_meeting>\t<http://rdf.freebase.com/ns/type.object.name>\t"Spotkanie w sprawie projektu"@pl\t.',
 '<http://rdf.freebase.com/ns/base.playgirlmanoftheyear.topic>\t<http://rdf.freebase.com/ns/

In [5]:
#osoby.count()
datalist = osoby.collect()

In [7]:
with open('people_data3.nt', 'w', encoding="utf-8") as f:
    f.writelines("%s\n" % item for item in datalist)

In [8]:
f.close()

In [9]:
sparql = SPARQLWrapper("http://localhost:3030/freebase/query")

In [10]:
meno_1 = input()

Fred Collard


In [11]:
meno_2 = input()

William Faulkner


In [12]:
print(meno_1, "  a  ", meno_2)

Fred Collard   a   William Faulkner


In [13]:
def najdi_datumy(meno):
    osoba = True
    dob = 0
    dod = 0
    name = 0
    sparql.setQuery(
    """
    PREFIX ns: <http://rdf.freebase.com/ns/>
    SELECT ?sub ?name (COALESCE(?alias_val, 0) as ?alias) (COALESCE(?birth_val, 0) AS ?birth) (COALESCE(?death_val, 0) as ?death)
    WHERE 
    {
       ?sub ns:type.object.name ?name .
       OPTIONAL {?sub ns:common.topic.alias ?alias_val} 
       OPTIONAL {?sub ns:people.person.date_of_birth ?birth_val} 
       OPTIONAL {?sub ns:people.deceased_person.date_of_death ?death_val} 
       FILTER (regex(?name, '""" + meno + """' , "i") || regex(?alias_val, '""" + meno + """' , "i"))
    }
    """
    )
    sparql.setReturnFormat('json')
    
    for result in sparql.query().convert()["results"]["bindings"]:
        #print(result)
        name = result["name"]["value"]
        dob = result["birth"]["value"].replace('-','')
        dod = result["death"]["value"].replace('-','')
            
        if dob == '0' and dod == '0':
            osoba = False
            break
            
        if 0 < int(dob) < 10000:
            dob = dob + '0101'  
        if 0 < int(dod) < 10000:
            dod = dod + '0101'
            
        if dob == '0':        
            dob = str(int(dod) - 1000000)
        if dod == '0':
            dod = str(int(dob) + 1000000)
        
        #print(name, dob, dod)
        if dob != '0' and dod != '0':
            break
    
    return dob, dod, osoba

In [14]:
def porovnanie(dob_1, dod_1, dob_2, dod_2):
    if dod_1 >= dob_2 and dob_1 <= dod_2:
        return True
    return False

In [15]:
def mohli_sa_stretnut(meno_1, meno_2):
    dob_1, dod_1, osoba_1 = najdi_datumy(meno_1)
    
    if osoba_1 == True:
        print(meno_1, datetime.datetime.strptime(dob_1, '%Y%m%d').date(), "-", datetime.datetime.strptime(dod_1, '%Y%m%d').date())
    
    dob_2, dod_2, osoba_2 = najdi_datumy(meno_2)
    
    if osoba_2 == True:
        print(meno_2, datetime.datetime.strptime(dob_2, '%Y%m%d').date(), "-", datetime.datetime.strptime(dod_2, '%Y%m%d').date())
    
    if osoba_1 == False or osoba_2 == False:
        print("Jedno alebo obe zadane mena nie su osoby alebo nemaju uvedene datumy narodenia ani umrtia")
    else:
        stretnutie = porovnanie(dob_1, dod_1, dob_2, dod_2)

        if stretnutie:
            print("Osoby sa mohli stretnut")
        else:
            print("Osoby sa nemohli stretnut")


In [18]:
mohli_sa_stretnut(meno_1, meno_2)

Fred Collard 1912-06-06 - 1986-12-01
William Faulkner 1897-09-25 - 1962-07-06
Osoby sa mohli stretnut
